In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 KB 3.6 MB/s eta 0:00:00


In [2]:
import os


import pandas as pd
import numpy as np
import random
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, StandardScaler , LabelEncoder
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

from keras.models import Model, load_model, save_model
#from keras.layers import Dense, Input, Dropout, Concatenate, Flatten, concatenate, Embedding, Reshape ,merge , dot
from tensorflow.keras.layers import BatchNormalization
from keras.regularizers import l2
from keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import make_column_selector as selector
import time
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras import regularizers
#import keras_tuner as kt
from tensorflow import keras

In [3]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

In [4]:
import tensorflow_recommenders as tfrs

In [7]:
# file paths
input1_path = '/content/user_features.csv'
input2_path = '/content/items_pre.csv'
input3_path = '/content/interactions_ranked.csv'

In [8]:
# loading datasets
users_df = pd.read_csv(input1_path)
songs_df = pd.read_csv(input2_path)
interactions_df = pd.read_csv(input3_path)

In [9]:
# Preprocessing 
def user_preprocess(dataframe):
    
    dff =  dataframe.copy()
    dff['MSISDN_VOICE'] = dff['MSISDN_VOICE'].astype('str')
 
    cateogry_columns=dff.select_dtypes(include=['object']).columns.tolist()
    integer_columns=dff.select_dtypes(include=['int64','float64']).columns.tolist()

    for column in dff:
        if dff[column].isnull().any():
                if(column in cateogry_columns):
                     dff[column]=dff[column].fillna(dff[column].mode()[0])
                else:
                     dff[column]=dff[column].fillna(dff[column].mode()[0])
    
    numerical_columns_selector = selector(dtype_exclude=object)
    categorical_columns_selector = selector(dtype_include=object)

    numerical_columns = numerical_columns_selector(dff)
    categorical_columns = categorical_columns_selector(dff)


    categorical_preprocessor = OrdinalEncoder()
    numerical_preprocessor = StandardScaler()
 
    preprocessor = ColumnTransformer([
    ('ordinal-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])
    
    data = preprocessor.fit_transform(dff)
    data = pd.DataFrame(data , columns = users_df.columns)
   
    
    return data

def item_preprocess(dataframe):
    
    dff =  dataframe.copy()
    dff['TONECODE'] = dff['TONECODE'].astype('str')
    dff['LANGUAGE'] = dff['LANGUAGE'].astype('str')
    dff['ARTIST']  = dff['ARTIST'].astype('str')
    
    cateogry_columns=dff.select_dtypes(include=['object']).columns.tolist()
    integer_columns=dff.select_dtypes(include=['int64','float64']).columns.tolist()

    for column in dff:
        if dff[column].isnull().any():
                if(column in cateogry_columns):
                     dff[column]=dff[column].fillna(dff[column].mode()[0])
                else:
                     dff[column]=dff[column].fillna(dff[column].mode()[0])
    
    numerical_columns_selector = selector(dtype_exclude=object)
    categorical_columns_selector = selector(dtype_include=object)

    numerical_columns = numerical_columns_selector(dff)
    categorical_columns = categorical_columns_selector(dff)


    categorical_preprocessor = OrdinalEncoder()
    numerical_preprocessor = StandardScaler()
 
    preprocessor = ColumnTransformer([
    ('ordinal-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])
    
    data = preprocessor.fit_transform(dff)
    data = pd.DataFrame(data , columns = songs_df.columns)
   
    
    return data

def interaction_preprocess(dataframe):
    dff = dataframe.copy()
    dff.drop(columns = ['RANKING' , 'NUM_OF_DAYS'] , axis = 1 , inplace = True)
    
    
    dff['MSISDN']   = dff['MSISDN'].astype('str')
    dff['TONECODE'] = dff['TONECODE'].astype('str')
    
    cateogry_columns=dff.select_dtypes(include=['object']).columns.tolist()
    integer_columns=dff.select_dtypes(include=['int64','float64']).columns.tolist()

    for column in dff:
        if dff[column].isnull().any():
                if(column in cateogry_columns):
                     dff[column]=dff[column].fillna(dff[column].mode()[0])
                else:
                     dff[column]=dff[column].fillna(dff[column].mode()[0])
    
    numerical_columns_selector = selector(dtype_exclude=object)
    categorical_columns_selector = selector(dtype_include=object)

    numerical_columns = numerical_columns_selector(dff)
    categorical_columns = categorical_columns_selector(dff)


    categorical_preprocessor = OrdinalEncoder()
    numerical_preprocessor = StandardScaler()
 
    preprocessor = ColumnTransformer([
    ('ordinal-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])
    
    data = preprocessor.fit_transform(dff)
    column_names = categorical_columns + numerical_columns
    data = pd.DataFrame(data , columns = column_names)
    data['RANKING'] = dataframe['RANKING']
    
    return data

In [13]:

processed_song = item_preprocess(songs_df)
processed_user = user_preprocess(users_df)
processed_interactions = interaction_preprocess(interactions_df)

In [14]:
df_train_test = processed_interactions.merge(processed_user , left_on=['MSISDN'] , right_on= ['MSISDN_VOICE'])
df_train_test = df_train_test.merge(processed_song , on = 'TONECODE')

In [15]:
df_train_test = df_train_test.drop_duplicates()


In [22]:
processed_interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236656 entries, 0 to 236655
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   MSISDN            236656 non-null  float64
 1   TONECODE          236656 non-null  float64
 2   TRANSACTION_DATE  236656 non-null  float64
 3   RANKING           236656 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 7.2 MB


In [16]:
df_train_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236496 entries, 0 to 240869
Data columns (total 37 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   MSISDN                     236496 non-null  float64
 1   TONECODE                   236496 non-null  float64
 2   TRANSACTION_DATE           236496 non-null  float64
 3   RANKING                    236496 non-null  int64  
 4   MSISDN_VOICE               236496 non-null  float64
 5   AGE                        236496 non-null  float64
 6   CONTACT_GENDER             236496 non-null  float64
 7   DISTRICT_CODE              236496 non-null  float64
 8   PREFERRED_LANGUAGE         236496 non-null  float64
 9   HSPA                       236496 non-null  float64
 10  ENTERTAINMENT_MUSIC_LOVER  236496 non-null  float64
 11  SOCIAL_MEDIA_USER          236496 non-null  float64
 12  ENTERTAINMENT_MOVIE_LOVER  236496 non-null  float64
 13  CYBER_GAMERS               23

In [20]:
user_columns = users_df.columns
item_columns  = songs_df.columns

In [21]:
num_user_features = len(user_columns)
num_item_features = len(item_columns)


In [23]:
item_train, item_test = train_test_split(df_train_test[item_columns], train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(df_train_test[user_columns], train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(df_train_test["RANKING"],    train_size=0.80, shuffle=True, random_state=1)
print(f"movie/item training data shape: {item_train.shape}")
print(f"movie/item test data shape: {item_test.shape}")

movie/item training data shape: (189196, 11)
movie/item test data shape: (47300, 11)


In [24]:
num_outputs = 32
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
     tf.keras.layers.Dense(256 , activation = "relu"),
      tf.keras.layers.Dense(128 , activation = "relu"),
      tf.keras.layers.Dense(num_outputs)
  
  
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
      tf.keras.layers.Dense(256 , activation = "relu"),
      tf.keras.layers.Dense(128 , activation = "relu"),
      tf.keras.layers.Dense(num_outputs)
  
  
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1)

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 11)]         0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           43168       ['input_1[0][0]']                
                                                                                                  
 sequential_1 (Sequential)      (None, 32)           40096       ['input_2[0][0]']                
                                                                                              

In [25]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)

In [26]:
tf.random.set_seed(1)
model.fit([user_train, item_train], y_train, epochs=30)

Epoch 1/30
5913/5913 [==============================] - 37s 6ms/step - loss: 20.7751
Epoch 2/30
5913/5913 [==============================] - 29s 5ms/step - loss: 20.7732
Epoch 3/30
5913/5913 [==============================] - 23s 4ms/step - loss: 20.7732
Epoch 4/30
5913/5913 [==============================] - 25s 4ms/step - loss: 20.7732
Epoch 5/30
5913/5913 [==============================] - 23s 4ms/step - loss: 20.7732
Epoch 6/30
5913/5913 [==============================] - 25s 4ms/step - loss: 20.7732
Epoch 7/30
5913/5913 [==============================] - 22s 4ms/step - loss: 20.7732
Epoch 8/30
5913/5913 [==============================] - 23s 4ms/step - loss: 20.7732
Epoch 9/30
5913/5913 [==============================] - 23s 4ms/step - loss: 20.7732
Epoch 10/30
5913/5913 [==============================] - 22s 4ms/step - loss: 20.7732
Epoch 11/30
5913/5913 [==============================] - 23s 4ms/step - loss: 20.7732
Epoch 12/30
5913/5913 [==============================] - 21s 4m

In [27]:
model.evaluate([user_test, item_test], y_test)

1479/1479 [==============================] - 3s 2ms/step - loss: 21.0527


21.052684783935547